# Distributionally robust optimization

This notebooks collects our exploration for the application of results from distributionally robust optimization (DRO) to calculte the worst-case expected maximum future value (EMAX).

The key reference at this point is the following:

* Zhaolin Hu, L. Jeff Hong (2013). [Kullback-Leibler divergence constrained distributionally robust optimizations](http://personal.cb.cityu.edu.hk/jeffhong/Papers/HuHong2013_technicalreport.pdf), technical report. 

The key result in this paper is **Theorem 1** where they establish that we can determine the worst-case value if the uncertainty set is the Kullback-Leibler divergence by simply solving the one-dimensional optimization problem.

\begin{align*}
\min_{\alpha \geq 0} \alpha \log E_{P_0}\left[\exp^{H(x, \xi) / \alpha}\right] + \alpha \eta
\end{align*}

**Structure**

* Example: Linear case with normal distirbution

* Comparison to discretization approach

* Comparing optimization algorithms

**TODO**

* We want to experiment with using IPOPT for the optimization.

* We want to improve the setup for the discretization approach which at first glance might actually be quite competitive in our setting where the evaluation of the expectation in the optimization step is quite costly.

## Example: Linear case with normal distribution

This code reproduces the example in the above manuscript presented in Section 5.2. Our application differs from this toy example only in the structure of the objective function $H(x, \xi)$. Here it is assumed to be linear $H(x, \xi) = \xi^T x$ whereas it is nonlinear in our case where  $H(x, \xi) = \max\{V_1(x, \xi_1), V_2(x, \xi_2), V_3(x, \xi_3), V_4(x, \xi_4), V_5(x, \xi_5)\}$.

This example has some nice feature for developing our production code:

* We can simply exchange the criterion function with our version.

* We can easily experiment with differnt problem dimsions.

* We can compare the performance of our numerical solution with the closed form solution. 

### Insights

* The quality of the numerical integration of the EMAX function is key in the performance (number of function evaluations required for the optimization) and quality (comparison to the closed-form solution).

* The one-dimensional optimization is very robust to choosing alternative optimization algorithms.

* We need to focus all effort on improving the quality of the numerical integration.

### Remarks

* The code below allows to investigate the impact of using Monte Carlo integration for the evaluation of the expectation as we also can simply comment in the closed-from solution for this ingredient into the optimization.

In [8]:
from functools import partial
from scipy.optimize import fminbound
import numpy as np

EPS_FLOAT = np.finfo(1.0).eps

    
def objective_function(x, eps):
    return np.dot(eps, x)
    
def evaluate_expectation_mc(model_spec, *args):
    def integrand(model_spec, alpha, eps):
        return np.exp(objective_function(model_spec['x'], eps) / alpha)
    alpha, eps_draws = args
    p_integrand = partial(integrand, model_spec, alpha)
    return np.mean(np.apply_along_axis(p_integrand, 1, eps_draws))

def evaluate_expectation_cf(model_spec, *args):
    alpha = args[0]
    arg1 = np.dot(model_spec['mean'], model_spec['x']) / alpha
    arg2 = np.dot(np.dot(model_spec['x'], model_spec['covs']), model_spec['x']) / (2 * alpha ** 2 )
    return np.exp(arg1 + arg2)
        
def convex_function(model_spec, eps_draws, alpha):
    expectation = evaluate_expectation(model_spec, alpha, eps_draws)
    rslt = alpha * np.log(expectation) + alpha * model_spec['eta']
    return rslt

def closed_form_solution(model_spec):
    arg1 = np.dot(model_spec['mean'], model_spec['x'])
    arg2 = np.sqrt(2.0 * model_spec['eta'])
    arg3 = np.sqrt(np.dot(np.dot(model_spec['x'], model_spec['covs']), model_spec['x']))
    return arg1 + arg2 * arg3

def get_worst_case_expectation(model_spec):
    """This is the wrapper"""
    np.random.seed(21)
    eps_draws = np.random.multivariate_normal(model_spec['mean'], model_spec['covs'], model_spec['num_draws'])

    lower, upper = 0.001, 5000
    rslt = fminbound(partial(convex_function, model_spec, eps_draws), lower, upper, xtol=EPS_FLOAT, full_output=True)

    return rslt


model_spec = dict()
model_spec['x'] = np.ones(1)
model_spec['mean'] = np.zeros(1)
model_spec['covs'] = np.identity(1)
model_spec['num_draws'] = 10000
model_spec['eta'] = 0.1

# We can use this code to either use the closed-form solution for the integration 
# of the expectation or do so by Monte Carlo integration. This allows to get a sense
# of how the simulation error affects the optimization step.
is_closed = False
if is_closed:
    evaluate_expectation = evaluate_expectation_cf
else:
    evaluate_expectation = evaluate_expectation_mc

print(get_worst_case_expectation(model_spec))
print(closed_form_solution(model_spec))

(2.2513399839578927, 0.4672106648335468, 0, 28)
0.4472135954999579


## Comparison to discretization approach

We want to compare the results from the more general approach above to the discretization approach we initially pursued. We will illustrate this the simple example of univariate standard normal distribution.

### Insights

* The results are rather close and the discretization approach is quite fast as it does not require to repeatedly solve the EMAX integral as in the approach above. We need to keep this in mind for some more extensive benchmarking at a later point. 

The key reference for this approach is:

* Arnab Nilim, Laurent El Ghaoui (2003). [Robust Control of Markov Decision Processes with
Uncertain Transition Matrices](https://people.eecs.berkeley.edu/~elghaoui/Pubs/RobMDP_OR2005.pdf), Operations Research, Vol. 53, No. 5, p.780 - 798.

The algorithm for discrete transitions is presented in Section 6.1 and implemented in the **robupy** package.

In [9]:
from robupy.auxiliary import get_worst_case_probs
from scipy.stats import norm

In [10]:
def get_worst_case_discretization(model_spec, num_points= 100000):
    np.random.seed(123)
    v = norm.rvs(size=num_points)

    q = np.tile(1.0, num_points) 
    q = q / np.sum(q)

    eta = model_spec['eta']
    p = get_worst_case_probs(v, q, eta)
    return np.matmul(v, p)

Now we can compare for a standard case.

In [11]:
model_spec = dict()
model_spec['x'] = np.ones(1)
model_spec['mean'] = np.zeros(1)
model_spec['covs'] = np.identity(1)
model_spec['num_draws'] = 1000
model_spec['eta'] = 0.1

Let's see how well the discretization performs.

In [12]:
print('discetization approach: {:5.3f}'.format(get_worst_case_discretization(model_spec)))
print('closed form: {:5.3f}'.format(closed_form_solution(model_spec)))

discetization approach: 0.448
closed form: 0.447


## Comparing optimization algorihtms

We compare different optimization algorithms to check for any differences in performance.

### Insights

* All algorithms pretty much find the same points but the derivative-based algorithsm do so with much less function evaluations. This appears true even though we only provide a noisy deriative that includes numerical and simulation error.

In [19]:
from scipy.optimize import approx_fprime
import nlopt

ALGO_LOCAL_LD = [nlopt.LD_MMA, nlopt.LD_SLSQP, nlopt.LD_LBFGS, nlopt.LD_TNEWTON_PRECOND_RESTART]
ALGO_LOCAL_LN = [nlopt.LN_COBYLA, nlopt.LN_PRAXIS, nlopt.LN_NELDERMEAD, nlopt.LN_SBPLX]

# Parameters for optimization
APPROX_FPRIME_EPS = 10e-6
NLOPT_XTOL_REL = 10e-5
NLOPT_MAXEVAL = 100

model_spec = dict()
model_spec['x'] = np.ones(1)
model_spec['mean'] = np.zeros(1)
model_spec['covs'] = np.identity(1)
model_spec['num_draws'] = 10000
model_spec['eta'] = 0.1

def convex_function_nlopt(model_spec, eps_draws, alpha, grad=np.array([])):
    if grad.size > 0:
        grad[0]  = approx_fprime(alpha, test_func, APPROX_FPRIME_EPS)
    expectation = evaluate_expectation(model_spec, alpha, eps_draws)
    rslt = alpha * np.log(expectation) + alpha * model_spec['eta']
    return float(rslt)

np.random.seed(21)
eps_draws = np.random.multivariate_normal(model_spec['mean'], model_spec['covs'], model_spec['num_draws'])
test_func = partial(convex_function_nlopt, model_spec, eps_draws)

for algo in ALGO_LOCAL_LN + ALGO_LOCAL_LD:
    
    opt = nlopt.opt(algo, 1)
    opt.set_lower_bounds([EPS_FLOAT])
    opt.set_min_objective(test_func)
    opt.set_xtol_rel(NLOPT_XTOL_REL)
    opt.set_maxeval(NLOPT_MAXEVAL)
    x = opt.optimize([0.5])
    
    info = list()
    info += [opt.get_algorithm_name(), x[0], opt.last_optimum_value()]
    info += [opt.get_numevals(), opt.last_optimize_result()]
    print('{:<105}{:10.5f}{:10.5f}{:5d}{:5d}'.format(*info))

COBYLA (Constrained Optimization BY Linear Approximations) (local, no-derivative)                           2.25129   0.46721   27    4
Principal-axis, praxis (local, no-derivative)                                                               2.25134   0.46721  100    5
Nelder-Mead simplex algorithm (local, no-derivative)                                                        2.25140   0.46721   30    4
Sbplx variant of Nelder-Mead (re-implementation of Rowan's Subplex) (local, no-derivative)                  2.25136   0.46721   49    4
Method of Moving Asymptotes (MMA) (local, derivative)                                                       2.25133   0.46721   13    4
Sequential Quadratic Programming (SQP) (local, derivative)                                                  2.25134   0.46721   10    4
Limited-memory BFGS (L-BFGS) (local, derivative-based)                                                      2.25134   0.46721   18    4
Preconditioned truncated Newton with restarting 